# per ogni input controllare che l'ufficio sia presente nella lista dei codici uffici

#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text:  # Check if the link is present in the whole body
                    compila = 'si'



                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [3]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result=''
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        rep = str(numero) + ' - ' + str(anno)
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': ufficio,
                            'natura': natura,
                            'ente': ente,
                            'repertorio': rep,
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number} , ente : {ente} , natura : {natura} , debitore : {debitore}')
        print(f"Error occurred: {e}")
    return results


#### sistemiamo l'input prima di mandare lo scaper 

In [2]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
import re
today = date.today()


In [5]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [6]:
input = pd.read_excel('Input/Raw/perimetro_03_06.xlsx')

In [7]:
list1 = ["TBS61575551",
        "AZCC20190203344",
        "CRCC103062611.001",
        "AZFT20230009230",
        #"EN912007140417_15042014"
        ]

input1 = input[input['NR_Contratto'].isin(list1)]

input2 = input[~input['N. Decreto'].isna()]
input2['N. RG'] = np.nan
input2['N° Repertorio'] = np.nan
input2['N. Cronologico'] = np.nan
input2['N.R.G.E PPT'] = np.nan
input2['N° Rep PPT'] = np.nan

input = pd.concat([input1, input2], ignore_index=True)


C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_12404\1751588785.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input2['N. RG'] = np.nan
C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_12404\1751588785.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input2['N° Repertorio'] = np.nan
C:\Users\eleonora.zarrilli\AppData\Local\Temp\ipykernel_12404\1751588785.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
input

In [8]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [ ]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [9]:
input = input.drop_duplicates()

In [ ]:


input = clean(input)
#input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
#input = input.drop(columns=['RICERCA IQERA'])

TOGLIERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [10]:
if input['N. RG'].notna().any():
        input['N. RG'] = (input['N. RG'].str.replace('Rg.', '', regex=False)
                    .str.replace('Nr.', '', regex=False)
                    .str.replace('N.', '', regex=False)
                    .str.strip()
                    )
        input[['nr.rg', 'year.rg']] = input['N. RG'].str.split('/', expand=True)

input['N. Decreto'] = (input['N. Decreto'] 
                            .str.replace('Nr.', '', regex=False)
                            .str.replace('Rg.', '', regex=False)
                            .str.replace('N.', '', regex=False)
                            .str.strip()
                        )
        
split_result = input['N. Decreto'].str.split('/', expand=True)
if split_result.shape[1] == 2:
            input[['nr.decreto', 'year.decreto']] = split_result
else:
            print("Split did not produce the expected number of columns for some rows.")
            # Handle the case here, e.g., by filling NaNs
            split_result = split_result.reindex(columns=[0, 1])
            input[['nr.decreto', 'year.decreto']] = split_result            

input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
input = input.drop(columns=['N. RG','N. Decreto','N° Repertorio','N. Cronologico','N.R.G.E PPT','N° Rep PPT','RICERCA IQERA'])

Split did not produce the expected number of columns for some rows.


In [11]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,ANNO DECRETO/PPT,Cliente,Esito Ricerca,nr.rg,year.rg,nr.decreto,year.decreto,tipo_ricerca
0,TBS61575551,CHOQUE AYMA NORMA CELIA,Tribunale,MILANO,2019,TIBERIUS,NaN,NaN,NaN,1517,2019,monitoria
1,AZCC20190203344,SCIMIA SABRINA,Giudice,L'AQUILA,2021,Azeta,NaN,169,2021,NaN,NaN,monitoria
2,CRCC103062611.001,CARZAGHI LUCIANA FEDERICA,Giudice,NOVARA,2022,MBCREDIT,NaN,689,2022,NaN,NaN,monitoria
3,AZFT20230009230,SAMA S A S DI ALFARANO ANTONIO C,Tribunale,TORINO,2015,Azeta,NaN,NaN,NaN,17223,2015,monitoria
4,CRS5602898,ALFANO EMILIO,Tribunale,SANTA MARIA CAPUA VETERE,2021,MBCREDIT,NaN,NaN,NaN,1681,2021,monitoria
...,...,...,...,...,...,...,...,...,...,...,...,...
1886,TBS40961578,MIGLIANO RAUL,Tribunale,NAPOLI,2019,TIBERIUS,NaN,NaN,NaN,9291,2019,monitoria
1887,TBS79674307,GRECO BRUNO,Tribunale,VELLETRI,2019,TIBERIUS,NaN,NaN,NaN,303,2019,monitoria
1888,UB13219464_15301332006,ANZANI MICHELE VANNI,Tribunale,LECCO,2021,Sirio,NaN,NaN,NaN,930,2021,monitoria
1889,XAZ4706119,COPETTI RODOLFO IND,Tribunale,PARMA,2021,Azeta,NaN,NaN,NaN,1595,2021,monitoria


In [12]:


mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',  # forse
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'"
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input
input = input.drop_duplicates()

In [13]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [14]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

input = input.drop_duplicates()

In [15]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

In [16]:
input['Debitore'] = (input['Debitore']
                     .str.replace('-', ' ')
                     .str.replace(r'\s+', ' ', regex=True)
                     .str.strip()
                    )


#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [17]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [18]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)


In [ ]:
input

In [19]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [20]:
input = input.drop(columns={'category'})
input['code_ente'] = input['code_ente'].apply(tuple)
input = input.drop_duplicates()
input['code_ente'] = input['code_ente'].apply(list)

In [ ]:
input.to_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',index=False)

In [21]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,nr,year
0,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1305,20
1,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1305,20
2,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1256,20
3,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1256,20
4,AA1200003174_06112018,TORTORA VINCENZA,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1290,20
...,...,...,...,...,...,...,...,...,...,...,...
2365,XUB385075,BUCCIONE PIERO,tribunale,chieti,Sirio,monitoria,TAH,chieti ut dpch,[001],365,2023
2366,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJ3,civitavecchia ut dprm2,[001],226,2023
2367,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJF,civitavecchia - ante 01.02.2015 ut dprm1,"[001, 004, 005, 006, 007, 011]",226,2023
2368,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",1975,2021


In [30]:
# new_df = input[2000:2500]

new_df = input[1000:]
# new_df = input[((input['Debitore'] == 'NOTARPIETRO MARIA GRAZIA') | (input['Debitore'] == 'ABBATE MICHELE')) &
#                    ((input['code_office'] == 'TET') | (input['code_office'] == 'TZL'))]

results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], 
                                    anno=row['year'],
                                    numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

Risultati in un df

In [31]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,MR12721395_15301118851,DEL RIO GIAN PAOLO,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 186 del 2021,GIUDICE DI PACE DI ORISTANO,50/22 - 2022,4MORI SARDEGNA S.R.L.,DELRIO,ATTO REGISTRATO,no
1,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,004,186 - 2021,-,-,-,no
2,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,005,186 - 2021,-,-,-,no
3,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,006,186 - 2021,-,-,-,no
4,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,007,186 - 2021,-,-,-,no
...,...,...,...,...,...,...,...,...,...,...
14094,XUB386298,PESCE MAURO,TAU,DI,006,1975 - 2021,-,-,-,no
14095,XUB386298,PESCE MAURO,TAU,AV,001,1975 - 2021,-,-,-,no
14096,XUB386298,PESCE MAURO,TAU,AV,005,1975 - 2021,-,-,-,no
14097,XUB386298,PESCE MAURO,TAU,AV,006,1975 - 2021,-,-,-,no


In [32]:
cliente = input[['NR_Contratto','Cliente']]
df = pd.merge(df,cliente,how='left', left_on='contratto',right_on='NR_Contratto' )
df.drop(columns=['NR_Contratto'], inplace=True)
df['Cliente'] = df['Cliente'].str.upper()
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
0,MR12721395_15301118851,DEL RIO GIAN PAOLO,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 186 del 2021,GIUDICE DI PACE DI ORISTANO,50/22 - 2022,4MORI SARDEGNA S.R.L.,DELRIO,ATTO REGISTRATO,no,4MORI
1,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,004,186 - 2021,-,-,-,no,4MORI
2,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,005,186 - 2021,-,-,-,no,4MORI
3,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,006,186 - 2021,-,-,-,no,4MORI
4,MR12721395_15301118851,DEL RIO GIAN PAOLO,TWK,DI,007,186 - 2021,-,-,-,no,4MORI
...,...,...,...,...,...,...,...,...,...,...,...
25557,XUB386298,PESCE MAURO,TAU,DI,006,1975 - 2021,-,-,-,no,SIRIO
25558,XUB386298,PESCE MAURO,TAU,AV,001,1975 - 2021,-,-,-,no,SIRIO
25559,XUB386298,PESCE MAURO,TAU,AV,005,1975 - 2021,-,-,-,no,SIRIO
25560,XUB386298,PESCE MAURO,TAU,AV,006,1975 - 2021,-,-,-,no,SIRIO


Righe con risultati

In [33]:
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
0,MR12721395_15301118851,DEL RIO GIAN PAOLO,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 186 del 2021,GIUDICE DI PACE DI ORISTANO,50/22 - 2022,4MORI SARDEGNA S.R.L.,DELRIO,ATTO REGISTRATO,no,4MORI
22,MRS37822658,SPIGA GIOVANNI,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 251 del 2022,GIUDICE DI PACE DI ORISTANO,46 - 2023,4MORI SARDEGNA S.R.L.,SPIGA,ATTO REGISTRATO,no,4MORI
34,MRS38131498,PIRAS SALVATORE,ORISTANO UT DPOR,DECRETO INGIUNTIVO n. 17 del 2023,TRIBUNALE DI ORISTANO,582 - 2023,4MORI SARDEGNA S.R.L.,PIRAS,ATTO REGISTRATO,no,4MORI
42,MRS38342305,COSSU DOMENICO,MILANO 1 UT DPMI1,DECRETO INGIUNTIVO n. 15545 del 2022,TRIB. CIVILE ORDINARIO,3760 - 2022,TORRETTA MATTEO,ASOLO SRL,STAMPATO AVVISO DI LIQUIDAZIONE,no,4MORI
80,MRS50460581,FADDA ANDREA,TEMPIO PAUSANIA UT DPSS,DECRETO INGIUNTIVO n. 15 del 2023,TRIBUNALE DI TEMPIO PAUSANIA,89/24 - 2024,4MORI SARDEGNA S.R.L.,FADDA,IN ATTESA DI PAGAMENTO,si,4MORI
...,...,...,...,...,...,...,...,...,...,...,...
25482,XUB379810,FICO DOMENICO ROSARIO,TORINO 1 UT DPTO1,DECRETO INGIUNTIVO n. 1145 del 2023,TRIBUNALE TORINO,4197 - 2024,SIRIO NPL S.R.L.,FICO,IN ATTESA DI PAGAMENTO,si,SIRIO
25511,XUB384584,BARUCCA VINCENZO,MACERATA UT DPMC,DECRETO INGIUNTIVO n. 599 del 2022,TRIBUNALE DI MACERATA,524/23 - 2023,SIRIO NPL S.R.L.,BARUCCA V.,ATTO REGISTRATO,no,SIRIO
25520,XUB384942,DIMITRI CARLO NICOLA,PESCARA UT DPPE,DECRETO INGIUNTIVO n. 930 del 2023,GIUDICE DI PACE DI PESCARA,263/24 - 2024,SIRIO NPL S.R.L.,DIMITRI CARLO NICOLA,IN ATTESA DI PAGAMENTO,si,SIRIO
25525,XUB385075,BUCCIONE PIERO,CHIETI UT DPCH,DECRETO INGIUNTIVO n. 365 del 2023,TRIBUNALE CHIETI,270 - 2024,PRELIOS CREDIT SOLUTIONS S.P.A,-,IN ATTESA DI PAGAMENTO,si,SIRIO


Guardare i nomi in format 'name +1'

In [34]:
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
919,SLCC201910546947000,MULE' VALENTINA,PORDENONE UT DPPN,DECRETO INGIUNTIVO n. 854 del 2023,GIUDICE DI PACE DI PORDENONE,306 - 2024,LEX SRL,MULþ VALENTINA + ALTRI,IN ATTESA DI PAGAMENTO,si,LEX
14595,UB13204818_15301322292,ORLANDI MARIO,MACERATA UT DPMC,DECRETO INGIUNTIVO n. 149 del 2023,TRIBUNALE DI MACERATA,1620/23 - 2023,SIRIO NPL S.R.L.,ORLANDI + 1,IN ATTESA DI PAGAMENTO,si,SIRIO
15097,UB13206047_15301323134,LIBERTI MARIO,PESARO UT DPPU,DECRETO INGIUNTIVO n. 844 del 2021,TRIBUNALE DI PESARO,121 - 2021,SIRIO NPL S.R.L.,LIBERTI MARIO +1,STAMPATO AVVISO DI LIQUIDAZIONE,no,SIRIO
15133,UB13206306_15301323303,VIOZZI LUCA,FERMO UT DPFM,DECRETO INGIUNTIVO n. 74 del 2022,TRIBUNALE DI FERMO,357 - 2023,SIRIO NPL S.R.L.,VIOZZI LUCA + 2,ATTO REGISTRATO,no,SIRIO
15880,UB13208139_15301324544,BERENATO ANDREA,PIACENZA UT DPPC,DECRETO INGIUNTIVO n. 658 del 2023,GIUDICE DI PACE DI PIACENZA,344/2024 - 2024,SIRIO NPL S.R.L. + 1,BERENATO ANDREA,IN ATTESA DI PAGAMENTO,si,SIRIO
16756,UB13210417_15301326044,NETNESS SOCIETA' A RESPONSABILITA' LIMITATA SE...,RAVENNA UT DPRA,DECRETO INGIUNTIVO n. 762 del 2023,TRIBUNALE DI RAVENNA,228/24 - 2024,SIRIO NPL S.R.L.,FOSCHI MORENO +1,ATTO REGISTRATO,no,SIRIO
17161,UB13211804_15301326985,BUCOSSI GIAMMARIO,MACERATA UT DPMC,DECRETO INGIUNTIVO n. 219 del 2022,TRIBUNALE DI MACERATA,218/23 - 2023,SIRIO NPL S.R.L.,BUCOSSI + 1,ATTO REGISTRATO,no,SIRIO
20094,UB13219341_15301331925,MARTIN RUBEN HECTOR,PESARO UT DPPU,DECRETO INGIUNTIVO n. 971 del 2021,TRIBUNALE DI PESARO,468 - 2021,SIRIO NPL S.R.L.,MARTIN RUBEN HECTOR + 1,IN ATTESA DI PAGAMENTO,si,SIRIO
21365,UB13224713_15301335338,CENTINI GIANCARLO,FERMO UT DPFM,DECRETO INGIUNTIVO n. 341 del 2022,TRIBUNALE DI FERMO,964 - 2023,SIRIO NPL S.R.L.,CENTINI FRANCO + 2,ATTO REGISTRATO,no,SIRIO
21437,UBS280818,IL GIGLIO SAS DI GIGLIO ANTONIO & C.,TREVISO UT DPTV,DECRETO INGIUNTIVO n. 2695 del 2023,TRIBUNALE TREVISO,1275/24 - 2023,SIRIO NPL S.R.L.,IL GIGLIO SAS DI GIGLIO +A+E+G,IN ATTESA DI PAGAMENTO,si,SIRIO


Toglie il +1 a chi lo ha

In [35]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()


In [36]:
# df = pd.DataFrame(results.explode().tolist())
df.to_excel(f'Output/{str(today)}_output_decreti2_v1.xlsx',index=False)


In [37]:
mask = df.apply(lambda row: (row['Cliente'] in row['attore']) or (row['attore'] in row['Cliente']) or (row['convenuto'] in row['debitore']), axis=1)

df[mask].to_excel(f'Output/{str(today)}_esiti_decreti2_v1.xlsx',index=False)

Uniamo tutti i file di output

In [3]:
import os

In [5]:
df = pd.DataFrame()
folder_path = 'Output'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'output' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel('Output/2024-06-07_output_totale.xlsx',index= False)
            

In [7]:
df = pd.DataFrame()
folder_path = 'Output'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') and 'esiti' in filename:
        file_path = os.path.join(folder_path, filename)
        dataframes.append(pd.read_excel(file_path))

if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

df = df.drop_duplicates()
df.to_excel('Output/2024-06-07_esiti_positivi.xlsx',index=False)